# BERTopic: Images only

In [1]:
import sys
sys.path.append('../../../')
import time
import pandas as pd

from src.helpers.postgres_helpers import execute_sql_select

## 1. Load prepared data from DB

In [2]:
table_name = "tg_oct_topic_modeling"
database = "telegram"

query = f"""SELECT * FROM {table_name}"""

data = execute_sql_select(command=query, database=database, return_result_as_df=True)
df = data.copy()

Column names:  ['index', 'channel_name', 'channel_id', 'message_id', 'text', 'cleaned_text', 'msg_date', 'media_id', 'msg_type', 'img_id', 'file_path', 'dhash', 'phash']
Connection to DB closed


In [3]:
len(df)

40110

## 2. Duplicate by cleaned_text + dhash pairs (base corpus)

In [5]:
# Deduplicate images by dhash 
df = df.drop_duplicates(['cleaned_text', 'dhash'])
len(df), df.columns

(40110,
 Index(['index', 'channel_name', 'channel_id', 'message_id', 'text',
        'cleaned_text', 'msg_date', 'media_id', 'msg_type', 'img_id',
        'file_path', 'dhash', 'phash'],
       dtype='object'))

## Initialize embedding and representation models

In [7]:
from bertopic import BERTopic
from bertopic.representation import VisualRepresentation
#from sentence_transformers import SentenceTransformer
from bertopic.backend import MultiModalBackend
from umap import UMAP


# Image embedding model
embedding_model = MultiModalBackend('clip-ViT-B-32', batch_size=32)

# Image to text representation model

image_to_text_model = "nlpconnect/vit-gpt2-image-captioning"
#image_to_text_model = "Salesforce/blip-image-captioning-base"
# image_to_text_model = "Salesforce/blip2-opt-2.7b" # quite large!
# image_to_text_model = "llava-hf/llava-1.5-7b-hf" # find out how to use with HF - maybe update transformers?

representation_model = {
    "Visual_Aspect": VisualRepresentation(image_to_text_model=image_to_text_model)
}

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


## 3. Train model


In [12]:
# Set seed in UMAP model for reproducibility
umap_model = UMAP(
    random_state=42
)

MIN_TOPIC_SIZES = [10, 20, 50]

for VAL in MIN_TOPIC_SIZES: 

    # Train our model with images only
    topic_model = BERTopic(
        embedding_model=embedding_model, 
        representation_model=representation_model,
        umap_model= umap_model,
        min_topic_size = VAL
    )

    print(f"Generating topics with min_topic_size = {str(VAL)}.")

    t0 = time.perf_counter()
    # Train model 
    topics, probs = topic_model.fit_transform(documents=None, images=images)#, embeddings=stored_embeddings)
    print("Time elapsed: ", time.strftime("%H:%M:%S", time.gmtime((time.perf_counter() - t0))))

    topic_info = topic_model.get_topic_info()
    print(f"{len(topic_info)} topics found.")
    print("Time elapsed: ", time.strftime("%H:%M:%S", time.gmtime((time.perf_counter() - t0))))

    # save model using safetensors
    version = f"v6_{str(VAL)}"
    model_name = f"bertopic_img_{version}"
    topic_model.save(f"models/{model_name}", serialization="safetensors", save_ctfidf=True)

    topic_info = topic_model.get_topic_info()
    topic_info.to_csv(f"bertopic_results/{model_name}_topic_info.csv")

    docs_topic_info = topic_model.get_document_info(images)
    docs_topic_info.to_csv(f"bertopic_results/{model_name}_docs_topic_info.csv")

Generating topics with min_topic_size = 10.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Time elapsed:  00:35:27
520 topics found.
Time elapsed:  00:35:27
Generating topics with min_topic_size = 20.


100%|██████████| 231/231 [00:27<00:00,  8.49it/s]


Time elapsed:  00:23:15
231 topics found.
Time elapsed:  00:23:15
Generating topics with min_topic_size = 50.


100%|██████████| 15/15 [00:01<00:00,  8.00it/s]


Time elapsed:  00:14:38
15 topics found.
Time elapsed:  00:14:38
